# Figures based on publicly available O3a data from GCNs
These are figures requested by Vicky for an upcoming talk

In [ ]:
import matplotlib
matplotlib.rcParams['figure.figsize'] = (10.0, 8.0)
matplotlib.rcParams['xtick.labelsize'] = 24.0
matplotlib.rcParams['ytick.labelsize'] = 24.0
matplotlib.rcParams['axes.titlesize'] = 27.0
matplotlib.rcParams['axes.labelsize'] = 27.0
matplotlib.rcParams['lines.markersize'] = 10.0
matplotlib.rcParams['legend.fontsize'] = 24.0
matplotlib.rcParams['text.usetex'] = True
matplotlib.rcParams['text.latex.preamble'] = r"\usepackage{amsmath}"
matplotlib.rcParams['font.serif'] = ['Computer Modern', 'Times New Roman']
matplotlib.rcParams['font.family'] = ['serif', 'STIXGeneral']
matplotlib.rcParams['legend.frameon'] = False
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

from astropy.cosmology import Planck15, z_at_value
from astropy import units

In [ ]:
O3a_events = {'S190408an': [False, 387, 1473, 358],
              'S190412m': [False, 156, 812, 194],
              'S190421ar': [False, 1444, 1628, 535],
              'S190425z': [True, 7461, 156, 41],
              'S190426c': [True, 1131, 377, 100],
              'S190503bf': [False, 448, 421, 105],
              'S190510g': [True, 1166, 227, 92],
              'S190512at': [False, 252, 1388, 322],
              'S190513bm': [False, 691, 1987, 501, 3],
              'S190517h': [False, 939, 2950, 1038, 3],
              'S190519bj': [False, 967, 3154, 791, 3],
              'S190521g': [False, 765, 3931, 953, 3],
              'S190521r': [False, 488, 1136, 279, 2],
              'S190602aq': [False, 1172, 797, 238, 3],
              'S190630ag': [False, 8493, 1059, 307, 2],
              'S190701ah': [False, 49, 1849, 446, 3],
              'S190706ai': [False, 826, 5263, 1402, 3],
              'S190707q': [False, 1375, 810, 234, 2],
              'S190720a': [False, 1599, 1159, 364, 2],
              'S190727h': [False, 151, 2839, 655, 3],
              'S190728q': [False, 104, 874, 171, 3],
              'S190814bv': [True, 23, 267, 52, 3],
              'S190828j': [False, 587, 1803, 423, 3],
              'S190828l': [False, 948, 1609, 426, 3],
              'S190901ap': [True, 13613, 242, 81, 2]
             } 


In [ ]:
events = pd.DataFrame.from_dict(O3a_events, orient='index',
    columns=['prob_bns', 'sky_area', 'dist', 'dist_unc', num_ifos])

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ymax = 2000

ns_count = 0
bh_count = 0


for event in O3a_events:
    prob_bns, sky_area, dist, dist_unc = O3a_events[event]
    
    label = None
    
    # Set BNS to a certain color
    if prob_bns:
        color = '#af8dc3'
        marker = 'o'
        ns_count += 1
        if ns_count == 2:
            label = 'High NS Probability'
    else:
        color = '#7fbf7b'
        bh_count += 1
        marker = 'd'
        if bh_count == 1:
            label = 'Low NS Probability'
            
    # Find redshift
    z = z_at_value(Planck15.luminosity_distance, dist * units.Mpc)
    
    # Set upper limit arrow if too high
    if sky_area > ymax:
#         plt.scatter(dist, ymax - 100, color=color, 
#             marker=marker, facecolors='none') #, mec='k', mfc='color')
        ax.annotate("", xy=(dist, ymax- 10), xytext=(dist, ymax-150),
            arrowprops=dict(arrowstyle="-|>", color=color))
        
    else:
        plt.errorbar(dist, sky_area, xerr=dist_unc, color=color, mec='k',
            marker=marker, label=label)

        
plt.legend(loc='upper right')
plt.xlim([0,7000])
plt.ylim([0, ymax])
plt.xlabel('Distance (Mpc)')
plt.ylabel('Sky Area (deg$^2$)')
        
        

# Set redshift ticks
ax1 = plt.twiny(ax)
z_ticks = np.linspace(0, 1, 6)
dist_ticks = [Planck15.luminosity_distance(z).value for z in z_ticks]
print(dist_ticks)
ax1.set_xticks(dist_ticks)
ax1.set_xticklabels(['%.1f' % v for v in z_ticks])


ax1.set_xlim(ax.get_xlim())
ax1.set_xlabel('Redshift')
        
#plt.savefig('O3a_dist_skyarea.png')
#plt.show()

# Color by number of interferometers

# Read in the data from GraceDb
Below, I tried to read in the data with a script instead of typing it in by hand.

This requires the following package: https://gw.readthedocs.io/projects/ligo-gracedb/en/latest/installation.html

In [ ]:
from ligo.gracedb.rest import GraceDb


# Initiate GraceDb client
client = GraceDb()

In [ ]:
# Search for public O3 events
event_iterator = client.superevents('1238112018 .. 1563214827 ADVOK')